In [1]:
import hatchet
import pandas as pd
from hatchet.frame import Frame
from hatchet.node import Node

In [2]:
file_location = '../../../hatchet/tests/data/gptl-cpu/log_cpu'

In [3]:

with open(file_location, 'r') as gptl_file:
    lines = gptl_file.readlines()
    start_index = 0
    stack = [None]
    for index, line in enumerate(lines):
        if line.strip().startswith('timer_name'):
            print(line)
            column_names = line.strip().split()
            lines = lines[index + 1:]
            break
    nodes = []
    root_graph_nodes = []
    for index, line in enumerate(lines):
        split_line = line.split()
        if len(split_line) == 0:
            lines = lines[:index]
            break
        while len(split_line) > len(column_names) + 1:
            split_line[1] = split_line[1] + ' ' + split_line[2]
            split_line.pop(2)
        
        node = {}
        # for i, column_name in enumerate(column_names):
        #     node[column_name] = split_line[i + 1]
        node['name'] = split_line[1]
        node['time'] = float(split_line[2])
        depth = int(split_line[0]) - 1
        # node['depth'] = depth
        
        if depth < len(stack) - 1:
            stack = stack[:depth + 1]
        
        
        frame_node = Frame(**node)
        graph_node = Node(frame_node, parent=stack[-1])

        # print('  ' * depth, 'name', node['name'], 'depth', depth)
        if(stack[-1] is not None):
            # print('\t parent', stack[-1].frame.get('name'))
            parent: Node = stack[-1]
            parent.add_child(graph_node)
        
        node['node'] = graph_node
        
        stack.append(graph_node)
        if(depth == 0):
            root_graph_nodes.append(graph_node)
        nodes.append(node)
        
    graph = hatchet.graphframe.Graph(root_graph_nodes)
    graph.enumerate_traverse()
    
    # print(column_names)
    # print(nodes)
    df = pd.DataFrame(nodes)
    df = df.set_index('node')
    gf = hatchet.GraphFrame(graph, df, [], ['time'])
    # print(len(root_graph_nodes))
    # display(df)
    

    timer_name                                            total       calls        min            max            avg      pct_tot   pct_par     par_eff



In [4]:
def dfs(node: Node, depth=0):
    print('->' * depth, node.frame.get('name'), node.frame.get('time'))
    for child in node.children:
        dfs(child, depth + 1)

dfs(root_graph_nodes[0])


 total time 40.55132
-> initialize 2.95966
->-> io_read 1.77514
->->-> analysis_bootstrap 8e-05
->->->-> init_read_mixedLayerDepths 2e-05
->-> reset_io_alarms 9e-05
->-> diagnostic solve 0.01847
->->-> equation of state 0.00239
->-> analysis_init 0.00019
->->-> init_mixedLayerDepths 0.00013
->-> SAL Init 0.0
-> analysis_compute_startup 0.62418
->-> compute_startup_mixedLayerDepths 0.0007
-> io_shortwave 0.00067
-> io_read 0.00122
-> reset_io_alarms 0.01092
-> analysis_precompute 0.00411
->-> precompute_mixedLayerDepths 0.00013
-> frazil 0.04386
-> land_ice_build_arrays 6e-05
-> submesoscale eddy velocity compute 0.21562
-> gm bolus velocity 4.03197
-> time integration 31.4201
->-> se timestep 31.41748
->->-> se prep 0.15367
->->-> se loop 24.22551
->->->-> se halo diag obd 0.00161
->->->-> se halo diag 0.00165
->->->-> se bcl vel 2.58672
->->->->-> se bcl vel tend 1.50499
->->->->->-> ocn_tend_vel 1.36265
->->->->->->-> bulk_ws 0.02625
->->->->->->-> coriolis 0.3234
->->->->->->-> vel 

In [5]:
display(gf.dataframe)

# print(nodes)

,name,time
node,,
"{'name': 'total time', 'time': '40.55132', 'type': 'None'}",total time,40.55132
"{'name': 'initialize', 'time': '2.95966', 'type': 'None'}",initialize,2.95966
"{'name': 'io_read', 'time': '1.77514', 'type': 'None'}",io_read,1.77514
"{'name': 'analysis_bootstrap', 'time': '8e-05', 'type': 'None'}",analysis_bootstrap,0.00008
"{'name': 'init_read_mixedLayerDepths', 'time': '2e-05', 'type': 'None'}",init_read_mixedLayerDepths,0.00002
...,...,...
"{'name': 'write_mixedLayerDepths', 'time': '0.00993', 'type': 'None'}",write_mixedLayerDepths,0.00993
"{'name': 'reset_alarm_mixedLayerDepths', 'time': '0.00611', 'type': 'None'}",reset_alarm_mixedLayerDepths,0.00611
"{'name': 'io_write', 'time': '0.61939', 'type': 'None'}",io_write,0.61939


In [6]:
df_2 = df.loc[df['name'] == 'total time']
display(df_2)

,name,time
node,,
"{'name': 'total time', 'time': '40.55132', 'type': 'None'}",total time,40.55132


In [7]:
print(gf.tree(metric_column='time'))

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v1.4.0

40.551 total time
├─ 0.068 analysis_compute
│  └─ 0.059 compute_mixedLayerDepths
├─ 0.624 analysis_compute_startup
│  └─ 0.001 compute_startup_mixedLayerDepths
├─ 0.000 analysis_finalize
│  └─ 0.000 finalize_mixedLayerDepths
├─ 0.004 analysis_precompute
│  └─ 0.000 precompute_mixedLayerDepths
├─ 0.003 analysis_restart
│  └─ 0.000 restart_mixedLayerDepths
├─ 0.020 analysis_write
│  ├─ 0.006 reset_alarm_mixedLayerDepths
│  └─ 0.010 write_mixedLayerDepths
├─ 0.044 frazil
├─ 4.032 gm bolus velocity
├─ 2.960 initialize
│  ├─ 0.000 SAL Init
│  ├─ 0.000 analysis_init
│  │  └─ 0.000 init_mixedLayerDepths
│  ├─ 0.018 diagnostic solve
│  │  └─ 0.002 equation of state
│  ├─ 1.775 io_read
│  │  └─ 0.000 analysis_bootstrap
│  │     └─ 0.000 init_read_mixedLayerDepths
│  └─ 0.000 reset_io_alarms
